In [ ]:
import torch
def expand(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'], st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'], st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

def expand_truncate(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'], st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'], st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.startswith('classifier.') or k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

def expand_three_times_truncate(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'],
         st_dict['layoutlmv3.embeddings.position_ids'],
         st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'],
         st_dict['layoutlmv3.embeddings.position_embeddings.weight'],
         st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    d = ['x', 'y', 'w', 'h']
    for dd in d: 
        st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'] = torch.cat(
            (st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'], 
             st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'])
            , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.startswith('classifier.') or k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

def expand_three_times_truncate_g(st_dict, new_path):
    st_dict['embeddings.position_ids'] = torch.cat(
        (st_dict['embeddings.position_ids'],
         st_dict['embeddings.position_ids'],
         st_dict['embeddings.position_ids'])
        , dim=1)
    st_dict['embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['embeddings.position_embeddings.weight'],
         st_dict['embeddings.position_embeddings.weight'],
         st_dict['embeddings.position_embeddings.weight'])
        , dim=0)
    # d = ['x', 'y', 'w', 'h']
    # for dd in d: 
    #     st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'] = torch.cat(
    #         (st_dict[f'embeddings.{dd}_position_embeddings.weight'], 
    #          st_dict[f'embeddings.{dd}_position_embeddings.weight'])
    #         , dim=0)
    print("expand position_ids to ", st_dict['embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.startswith('classifier.') or k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

In [ ]:
from layoutlmft.models.layoutlmv3.modeling_layoutlmv3 import LayoutLMv3ForRelationExtraction, LayoutLMv3ForTokenClassification, LayoutLMv3Model
pt_514 = LayoutLMv3Model.from_pretrained(
    'layoutlmv3-base'
)

In [ ]:
pt_514_d = pt_514.state_dict()
expand_truncate(pt_514_d, "layoutlmv3-large-ner-1028/pytorch_model.bin")

In [ ]:
pt_514_d = pt_514.state_dict()
expand_three_times_truncate_g(pt_514_d, "layoutlmv3-base-1542/pytorch_model.bin")

In [6]:
import layoutlmft.models.layoutlmv3
from layoutlmft.models.layoutlmv3.modeling_layoutlmv3 import LayoutLMv3ForRelationExtraction, LayoutLMv3ForTokenClassification, LayoutLMv3Model
from transformers import AutoConfig

config_1028 = AutoConfig.from_pretrained(
    'layoutlmv3-base-1028'
)
pt_base_1028 = LayoutLMv3ForTokenClassification.from_pretrained(
    'layoutlmv3-base-1028',
    config=config_1028
)
config_1542 = AutoConfig.from_pretrained(
    'layoutlmv3-base-1542'
)
pt_base_1542 = LayoutLMv3ForTokenClassification.from_pretrained(
    'layoutlmv3-base-1542',
    config=config_1542
)
config_large_1028 = AutoConfig.from_pretrained(
    'layoutlmv3-large-1028'
)
pt_large_1028 = LayoutLMv3ForTokenClassification.from_pretrained(
    'layoutlmv3-large-1028',
    config=config_large_1028
)
config_large_1542 = AutoConfig.from_pretrained(
    'layoutlmv3-large-1542'
)
pt_large_1542 = LayoutLMv3ForTokenClassification.from_pretrained(
    'layoutlmv3-large-1542',
    config=config_large_1542
)



Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at layoutlmv3-base-1028 and are newly initialized: ['layoutlmv3.encoder.layer.2.attention.self.lam', 'classifier.weight', 'layoutlmv3.encoder.layer.6.attention.self.lam', 'layoutlmv3.encoder.layer.8.attention.self.lam', 'layoutlmv3.encoder.layer.4.attention.self.lam', 'layoutlmv3.encoder.layer.5.attention.self.lam', 'layoutlmv3.encoder.layer.0.attention.self.lam', 'layoutlmv3.encoder.layer.9.attention.self.lam', 'layoutlmv3.encoder.layer.10.attention.self.lam', 'layoutlmv3.encoder.layer.3.attention.self.lam', 'layoutlmv3.encoder.layer.7.attention.self.lam', 'layoutlmv3.encoder.layer.11.attention.self.lam', 'classifier.bias', 'layoutlmv3.encoder.layer.1.attention.self.lam']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at layoutlmv3-b

In [5]:
assert len(pt_base_1028.state_dict()) == len(pt_base_1542.state_dict())
for (k1, v1),(k2, v2) in zip(pt_base_1028.state_dict().items(), pt_base_1542.state_dict().items()):
    print(k1, v1.shape, k2, v2.shape)

layoutlmv3.cls_token torch.Size([1, 1, 768]) layoutlmv3.cls_token torch.Size([1, 1, 768])
layoutlmv3.pos_embed torch.Size([1, 197, 768]) layoutlmv3.pos_embed torch.Size([1, 197, 768])
layoutlmv3.embeddings.position_ids torch.Size([1, 1028]) layoutlmv3.embeddings.position_ids torch.Size([1, 1542])
layoutlmv3.embeddings.word_embeddings.weight torch.Size([50265, 768]) layoutlmv3.embeddings.word_embeddings.weight torch.Size([50265, 768])
layoutlmv3.embeddings.token_type_embeddings.weight torch.Size([1, 768]) layoutlmv3.embeddings.token_type_embeddings.weight torch.Size([1, 768])
layoutlmv3.embeddings.LayerNorm.weight torch.Size([768]) layoutlmv3.embeddings.LayerNorm.weight torch.Size([768])
layoutlmv3.embeddings.LayerNorm.bias torch.Size([768]) layoutlmv3.embeddings.LayerNorm.bias torch.Size([768])
layoutlmv3.embeddings.position_embeddings.weight torch.Size([1028, 768]) layoutlmv3.embeddings.position_embeddings.weight torch.Size([1542, 768])
layoutlmv3.embeddings.x_position_embeddings.weig

In [7]:
assert len(pt_large_1028.state_dict()) == len(pt_large_1542.state_dict())
for (k1, v1),(k2, v2) in zip(pt_large_1028.state_dict().items(), pt_large_1542.state_dict().items()):
    print(k1, v1.shape, k2, v2.shape)

layoutlmv3.cls_token torch.Size([1, 1, 1024]) layoutlmv3.cls_token torch.Size([1, 1, 1024])
layoutlmv3.pos_embed torch.Size([1, 197, 1024]) layoutlmv3.pos_embed torch.Size([1, 197, 1024])
layoutlmv3.embeddings.position_ids torch.Size([1, 1028]) layoutlmv3.embeddings.position_ids torch.Size([1, 1542])
layoutlmv3.embeddings.word_embeddings.weight torch.Size([50265, 1024]) layoutlmv3.embeddings.word_embeddings.weight torch.Size([50265, 1024])
layoutlmv3.embeddings.token_type_embeddings.weight torch.Size([1, 1024]) layoutlmv3.embeddings.token_type_embeddings.weight torch.Size([1, 1024])
layoutlmv3.embeddings.LayerNorm.weight torch.Size([1024]) layoutlmv3.embeddings.LayerNorm.weight torch.Size([1024])
layoutlmv3.embeddings.LayerNorm.bias torch.Size([1024]) layoutlmv3.embeddings.LayerNorm.bias torch.Size([1024])
layoutlmv3.embeddings.position_embeddings.weight torch.Size([1028, 1024]) layoutlmv3.embeddings.position_embeddings.weight torch.Size([1542, 1024])
layoutlmv3.embeddings.x_position_e